# Projeto 2 - Ciência dos Dados

Nome: Beatriz Lourenço

Nome: Gabriel Yamashita

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [91]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [92]:
import tweepy
import math
import os.path
import pandas as pd
import json
import re 
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@Reme72277305***

In [93]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [94]:
#Produto escolhido:
produto = "McDonalds"

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [95]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break
# shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [96]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [97]:
data = pd.read_excel("McDonalds.xlsx",sheet_name = "Treinamento")
colunas = ["Treinamento", "relevancia"]
data = data.loc[:,colunas]

def cleanup(text):
    punctuation = "[!\-.:?;,'#-@]"
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [98]:
rel = " ".join(data[data["relevancia"] == 1].Treinamento)
irrel = " ".join(data[data["relevancia"] == 0].Treinamento)

rel = cleanup(rel)
irrel = cleanup(irrel)

rel = pd.Series(rel.split())
irrel = pd.Series(irrel.split())

In [99]:
rel_value = rel.value_counts(True)
rel_value

mcdonalds_br      0.040491
e                 0.034356
de                0.033129
o                 0.026994
no                0.023313
que               0.023313
mcdonalds         0.019632
do                0.018405
ifood             0.017178
eu                0.014724
a                 0.014724
não               0.013497
um                0.012270
nunca             0.012270
é                 0.012270
uma               0.008589
tem               0.008589
da                0.007362
https             0.007362
t                 0.007362
co                0.007362
pelo              0.006135
comer             0.006135
só                0.006135
pedido            0.006135
rt                0.004908
pedi              0.004908
mista             0.004908
se                0.004908
mais              0.004908
                    ...   
vir               0.001227
abraço            0.001227
hj                0.001227
tirado            0.001227
👍🏻                0.001227
mecôzo            0.001227
l

In [100]:
irrel_value = irrel.value_counts(True)
irrel_value

mcdonalds         0.035303
de                0.028193
mcdonalds_br      0.025251
que               0.020103
e                 0.017897
o                 0.017651
do                0.017406
a                 0.015445
eu                0.014955
t                 0.013974
é                 0.013974
co                0.013729
não               0.013729
https             0.013729
rt                0.013729
no                0.013239
um                0.012748
com               0.010051
em                0.008581
uma               0.007110
pra               0.006864
da                0.006864
por               0.006374
na                0.005884
q                 0.005393
minha             0.004903
se                0.004413
você              0.004413
comer             0.004168
me                0.004168
                    ...   
facissu           0.000245
arrependida       0.000245
mascote           0.000245
trabalha          0.000245
bruniin           0.000245
invenção          0.000245
d

___

In [103]:
total = rel + irrel
total = total.value_counts(True)
total

mcdonalds_brdo           0.004908
nãomcdonalds             0.003681
domcdonalds_br           0.002454
mcdonalds_brir           0.002454
queque                   0.002454
mcdonalds_brnão          0.002454
ode                      0.002454
temde                    0.002454
nuncamcdonalds           0.002454
mcdonalds_bre            0.002454
mcdonaldsrt              0.002454
euhttps                  0.002454
depor                    0.002454
mcdonalds_brmcdonalds    0.002454
foodpizza                0.001227
ufpra                    0.001227
ahambúrgueres            0.001227
cupommcdonalds_br        0.001227
bomchamando              0.001227
fazermcdo                0.001227
ofazendo                 0.001227
pedirem                  0.001227
dono                     0.001227
eudos                    0.001227
okkkkkkkk                0.001227
aumentoutá               0.001227
nopor                    0.001227
doalgo                   0.001227
fazernotinha             0.001227
cort          

In [104]:
inter_rel = set(rel_value.index)
inter_irrel = set(irrel_value.index)
inter = inter_rel.intersection(inter_irrel)
inter

{'a',
 'acabei',
 'achei',
 'acho',
 'agora',
 'agr',
 'ai',
 'algo',
 'anos',
 'ao',
 'apenas',
 'app',
 'aqui',
 'as',
 'assim',
 'até',
 'aí',
 'bacon',
 'batatas',
 'big',
 'bk',
 'bom',
 'burgerkingbr',
 'cabeça',
 'caixa',
 'cara',
 'casquinha',
 'chocolate',
 'co',
 'com',
 'combo',
 'comendo',
 'comer',
 'comida',
 'como',
 'compra',
 'cupom',
 'da',
 'dar',
 'das',
 'de',
 'demais',
 'devo',
 'dia',
 'dinheiro',
 'diz',
 'do',
 'domingo',
 'dá',
 'e',
 'ele',
 'eles',
 'em',
 'errado',
 'erro',
 'especial',
 'esse',
 'estão',
 'eu',
 'experiência',
 'falando',
 'falta',
 'fanchapwr',
 'fast',
 'fazer',
 'feliz',
 'fica',
 'fomos',
 'food',
 'frio',
 'fui',
 'gente',
 'h',
 'hj',
 'hoje',
 'https',
 'i',
 'ifood',
 'já',
 'kkkk',
 'la',
 'lanche',
 'loja',
 'lugar',
 'lá',
 'mais',
 'manauara',
 'mandar',
 'mas',
 'mc',
 'mcdonald',
 'mcdonalds',
 'mcdonalds_br',
 'me',
 'meia',
 'melhor',
 'mensagem',
 'meu',
 'milk',
 'milkshake',
 'minha',
 'mt',
 'muito',
 'na',
 'nada',
 '

___

In [118]:
alpha = 1
V = 1e6

rel_value_number = rel.value_counts()
sum_rel_value_number = rel_value_number.sum()

irrel_value_number = irrel.value_counts()
sum_irrel_value_number = irrel_value_number.sum()

prob_p_rel = (rel_value_number + 1)/(sum_rel_value_number + alpha*V)
prob_p_irrel = (irrel_value_number + 1)/(sum_irrel_value_number + alpha*V)

prob_p_desc_rel = alpha / (sum_rel_value_number + alpha*V)
prob_p_desc_irrel = alpha / (sum_irrel_value_number + alpha*V)

prob_p_rel.sort_values(ascending=False)

e               0.000027
o               0.000021
mcdonalds_br    0.000019
de              0.000013
mcdonalds       0.000012
mais            0.000012
que             0.000012
um              0.000011
não             0.000010
tá              0.000010
a               0.000010
no              0.000009
pra             0.000009
https           0.000008
t               0.000008
com             0.000008
do              0.000008
co              0.000008
ifood           0.000007
é               0.000007
da              0.000007
vez             0.000006
uma             0.000006
em              0.000006
na              0.000006
pelo            0.000005
compro          0.000005
molho           0.000005
pedido          0.000005
eles            0.000005
                  ...   
👍👍              0.000002
gasto           0.000002
isso            0.000002
unidade         0.000002
porcaria        0.000002
vacilo          0.000002
aumentando      0.000002
caloteiro       0.000002
pedindo         0.000002


In [109]:
rel_value_number = rel.value_counts()
irrel_value_number = irrel.value_counts()
total_number = irrel_value_number +irrel_value_number

prob_p_rel = (rel_value + 1)/(rel_value_number + total_number)
prob_p_irrel = (irrel_value + 1)/(irrel_value_number + total_number)

prob_p_rel.sort_values(ascending=False)

meia               0.333742
especial           0.333742
super              0.333742
cupom              0.333742
sinto              0.333742
shake              0.333742
demais             0.333742
sanduíche          0.333742
milk               0.333742
diz                0.333742
rio                0.333742
refri              0.333742
eles               0.333742
quasequeeucaio_    0.333742
errado             0.333742
experiência        0.333742
chocolate          0.333742
falando            0.333742
fanchapwr          0.333742
fast               0.333742
fica               0.333742
food               0.333742
onde               0.333742
oi                 0.333742
la                 0.333742
lugar              0.333742
nas                0.333742
mandar             0.333742
melhor             0.333742
mt                 0.333742
                     ...   
❤️                      NaN
🍔🍟                      NaN
🏟                       NaN
👍🏻                      NaN
💔                   

In [110]:
prob_p_irrel.sort_values(ascending=False)

arielsis        0.333415
dew             0.333415
🤣🤣🤣             0.333415
senhor          0.333415
putaqueparil    0.333415
comentários     0.333415
demos           0.333415
desses          0.333415
também          0.333415
yvlfp           0.333415
tags            0.333415
ey              0.333415
quebrou         0.333415
indo            0.333415
reggae          0.333415
onde            0.333415
nuludao         0.333415
pouk            0.333415
dúvida          0.333415
bob’s           0.333415
prefiro         0.333415
mec             0.333415
atnekmc         0.333415
favor           0.333415
neto            0.333415
detox           0.333415
queres          0.333415
servidos        0.333415
horas           0.333415
talvez          0.333415
                  ...   
me              0.019690
comer           0.019690
você            0.018600
se              0.018600
minha           0.016748
q               0.015233
na              0.013971
por             0.012902
da              0.011986


In [111]:
rel_n = " ".join(data[data["relevancia"] == 1].Treinamento).count()
irrel_n = " ".join(data[data["relevancia"] == 0].Treinamento).count()

prob_rel = rel_n/(rel_n + irrel_n)
prob_irrel = irrel_n/(rel_n + irrel_n)

print(prob_rel)
print(prob_irrel)

TypeError: count() takes at least 1 argument (0 given)

In [ ]:
prob_rel_p = prob_p_rel*prob_rel
prob_irrel_p = prob_p_irrel*prob_irrel

___

In [114]:
data = pd.read_excel("McDonalds.xlsx",sheet_name = "Teste")
colunas = ["Teste", "relevancia"]
data = data.loc[:,colunas]

rel = " ".join(data[data["relevancia"] == 1].Teste)
irrel = " ".join(data[data["relevancia"] == 0].Teste)

rel = cleanup(rel)
irrel = cleanup(irrel)

rel = pd.Series(rel.split())
irrel = pd.Series(irrel.split())

In [125]:
w = cleanup(data.iloc[100]['Teste']).split()

In [140]:
import numpy as np
np.log(prob_p_rel.reindex(w).fillna(prob_p_desc_rel)).sum()

-369.0582944808211

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**